In [ ]:
%%capture

!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore') 
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# %cd '/content/gdrive/MyDrive/SASH/'

/content/gdrive/MyDrive/SASH


In [ ]:
nse_Symbol=['ADANIPORTS.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJAJFINSV.NS',
 'BAJFINANCE.NS',
 'BHARTIARTL.NS',
 'BPCL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DIVISLAB.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFC.NS',
 'HDFCBANK.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'IOC.NS',
 'ITC.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBILIFE.NS',
 'SBIN.NS',
 'SHREECEM.NS',
 'SUNPHARMA.NS',
 'TATACONSUM.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TCS.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'UPL.NS',
 'WIPRO.NS']

In [ ]:
# nse=pd.read_csv('/content/gdrive/MyDrive/SASH/Nifty50.csv')
# nse['Symbol']=nse['Symbol'] + '.NS'

In [ ]:
# nse=pd.read_csv('ALL_FNO.csv')

# BACK TEST

In [ ]:
def checker(stock_name,start_date,end_date):
  #**Condition 1**
  #1st 15min high low should get break. That means days's high and day's low should be greater and lesser than 15min 1st candle high and low.
  df1 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    interval='15m')
  df1.reset_index(inplace=True)
  df1=df1.dropna().reset_index(drop=True)
  df1['Datetime']=df1['Datetime'].astype('str')
  # new
  olddf=pd.DataFrame(columns=['Datetime','Open','High','Low','Close','Checking High','Checking Low'])
  for i in range(0,len(df1),25):
    newdf=df1[i:i+25:]
    newdf['Checking High'] = ['Yes' if newdf['High'][i] == max(newdf['High']) else 'No' for i in range(i,i+25)]
    newdf['Checking Low'] = ['Yes' if newdf['Low'][i] == min(newdf['Low']) else 'No' for i in range(i,i+25)]
    olddf=olddf.append(newdf,ignore_index=True)
  # check inside
  Insider=[]
  for i in range(0,len(olddf)):
    try:
      if ((olddf['High'][i] > olddf['High'][i+1] and olddf['High'][i] > olddf['High'][i+2] and olddf['High'][i] > olddf['High'][i+3]) and 
          (olddf['Low'][i] < olddf['Low'][i+1] and olddf['Low'][i] < olddf['Low'][i+2] and olddf['Low'][i] < olddf['Low'][i+3])):
        Insider.append('Yes')
      else:
        Insider.append('No')
    except:
      Insider.append('No')
  olddf['Insider'] = Insider
  df1=olddf[olddf['Datetime'].str.contains('09:15')].reset_index(drop=True)
  df1=df1.drop_duplicates().reset_index(drop=True)
  #Now check the days high and low are greater or not than 15min 1st candle high and low
  df2 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    progress=False)
  df2.reset_index(inplace=True)
  df2=df2.dropna().reset_index(drop=True)
  # new - correction of high low
  df2['15Min High']= df1['High']
  df2['15Min Low']= df1['Low']
  df2['Checking High']= df1['Checking High']
  df2['Checking Low']= df1['Checking Low']
  df2['Insider']=df1['Insider']
  df2['15Min High'] = [df2['High'][i] if df2['Checking High'][i]=='Yes' else df2['15Min High'][i] for i in range(0,len(df2)) ]
  df2['15Min Low'] = [df2['Low'][i] if df2['Checking Low'][i]=='Yes' else df2['15Min Low'][i] for i in range(0,len(df2)) ]
  df2=df2.drop(['Checking High','Checking Low'],axis=1)
  df2['15min HL Break'] = ['Yes' if (df2['High'][i] > df2['15Min High'][i]  and df2['Low'][i] < df2['15Min Low'][i]) else 'No' for i in range(0,len(df2))]
  df2['15min HL Break']=df2['15min HL Break'].shift(1)
  df2=df2[df2['15min HL Break'] == 'Yes'].reset_index(drop=True)
  df2=df2[df2['Insider'] == 'Yes'].reset_index(drop=True)
  return (list(df2['Open']),list(df2['Close']),list(df2['Date']))

In [ ]:
%%capture

# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(nse_Symbol):
    selected_stocks[i]=checker(i,'2021-06-24','2021-07-05')
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')
final['Stock']='NSE:' + final['Stock'].str.strip('.NS') + ','


In [ ]:
# final.to_excel('3June.xlsx',index=False)
# files.download('3June.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
final

,Stock,Date
0,"NSE:AXISBANK,",2021-07-02
1,"NSE:BAJAJ-AUTO,",2021-07-01
2,"NSE:BHARTIARTL,",2021-06-30
3,"NSE:CIPLA,",2021-06-28
4,"NSE:DIVISLAB,",2021-06-25
5,"NSE:INDUSINDBK,",2021-07-01
6,"NSE:INFY,",2021-06-30
7,"NSE:JSWSTEEL,",2021-07-02
8,"NSE:KOTAKBANK,",2021-06-28
9,"NSE:LT,",2021-06-28


# NEXT DAY

In [ ]:
def checker(stock_name,start_date,end_date):
  #**Condition 1**
  #1st 15min high low should get break. That means days's high and day's low should be greater and lesser than 15min 1st candle high and low.
  df1 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    interval='15m')
  df1.reset_index(inplace=True)
  df1=df1.dropna().reset_index(drop=True)
  df1['Datetime']=df1['Datetime'].astype('str')
  # new - add checking condition if highest or lowest
  olddf=pd.DataFrame(columns=['Datetime','Open','High','Low','Close','Checking High','Checking Low'])
  for i in range(0,len(df1),25):
    newdf=df1[i:i+25:]
    newdf['Checking High'] = ['Yes' if newdf['High'][i] == max(newdf['High']) else 'No' for i in range(i,i+25)]
    newdf['Checking Low'] = ['Yes' if newdf['Low'][i] == min(newdf['Low']) else 'No' for i in range(i,i+25)]
    olddf=olddf.append(newdf,ignore_index=True)
  df1=olddf[olddf['Datetime'].str.contains('09:15')].reset_index(drop=True)
  df1=df1.drop_duplicates().reset_index(drop=True)
  #Now check the days high and low are greater or not than 15min 1st candle high and low
  df2 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    progress=False)
  df2.reset_index(inplace=True)
  df2=df2.dropna().reset_index(drop=True)
  # new - correction of high low
  df2['15Min High']= df1['High']
  df2['15Min Low']= df1['Low']
  df2['Checking High']= df1['Checking High']
  df2['Checking Low']= df1['Checking Low']
  df2['15Min High'] = [df2['High'][i] if df2['Checking High'][i]=='Yes' else df2['15Min High'][i] for i in range(0,len(df2)) ]
  df2['15Min Low'] = [df2['Low'][i] if df2['Checking Low'][i]=='Yes' else df2['15Min Low'][i] for i in range(0,len(df2)) ]
  df2=df2.drop(['Checking High','Checking Low'],axis=1)
  # continue
  df2['15min HL Break'] = ['Yes' if (df2['High'][i] > df2['15Min High'][i]  and df2['Low'][i] < df2['15Min Low'][i]) else 'No' for i in range(0,len(df2))]
  #**condition2**
  df2=df2[-1:]
  df2=df2[df2['15min HL Break'] == 'Yes'].reset_index(drop=True)
  df2['Date']=df2['Date'].astype('str')
  return (list(df2['Close']),list(df2['Date']))

In [ ]:

def checker2(stock_name,start_date,end_date):
  df1 = yf.download(stock_name, 
                    start=start_date, 
                    end=end_date,
                    interval='15m')
  df1.reset_index(inplace=True)
  df1=df1.dropna().reset_index(drop=True)
  df1['Datetime']=df1['Datetime'].astype('str')
  Insider=[]
  for i in range(0,len(df1)):
    try:
      if ((df1['High'][i] > df1['High'][i+1] and df1['High'][i] > df1['High'][i+2] ) and 
          (df1['Low'][i] < df1['Low'][i+1] and df1['Low'][i] < df1['Low'][i+2] )):
        Insider.append('Yes')
      else:
        Insider.append('No')
    except:
      Insider.append('No')
  df1['Insider'] = Insider
  df1=df1[df1['Datetime'].str.contains('09:15')].reset_index(drop=True)
  df1=df1[df1['Insider'] == 'Yes']
  df1=df1.drop_duplicates().reset_index(drop=True)
  return (list(df1['Close']),list(df1['Datetime']))

In [ ]:
dold='2021-07-01'
dtod='2021-07-05'
dtom='2021-07-06'

In [ ]:
%%capture

# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(nse['Symbol']):
    selected_stocks[i]=checker(i,dold,dtod)
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
# print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')

# 2nd filtering
# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(final['Stock']):
    selected_stocks[i]=checker2(i,dtod,dtom)
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
# print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')


In [ ]:
final

,Stock,Date
0,BHARTIARTL.NS,2021-07-05 09:15:00+05:30
1,EXIDEIND.NS,2021-07-05 09:15:00+05:30
2,HDFC.NS,2021-07-05 09:15:00+05:30
3,LICHSGFIN.NS,2021-07-05 09:15:00+05:30
4,NAUKRI.NS,2021-07-05 09:15:00+05:30
5,ULTRACEMCO.NS,2021-07-05 09:15:00+05:30
6,ZEEL.NS,2021-07-05 09:15:00+05:30
7,PAGEIND.NS,2021-07-05 09:15:00+05:30
8,MCDOWELL-N.NS,2021-07-05 09:15:00+05:30
